[WordAndDoc2vec](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/WordAndDoc2vec)

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
from feature_eng import lowcols

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.jpeg'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name, pil_kwargs={'quality': 100})

# load sample data

In [ ]:
X_df = pd.read_csv('../input/sample-data-wordanddoc2vec/sample010.csv', index_col=0)
X_df_user_group = pd.read_csv('../input/sample-data-wordanddoc2vec/sample010_user_group.csv', index_col=0)
X_df_brand_group = pd.read_csv('../input/sample-data-wordanddoc2vec/sample010_brand_group.csv', index_col=0)
X_df

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(X_df.values.T)
save_img()

# WordAndDoc2vec

In [ ]:
wd2v = lowcols.WD2vec(X_df)
wd2v

In [ ]:
# num_features = 2

# models = wd2v.make_model(num_features=num_features, seed=101, embeddings_val=0.2)

# print('\n\n##################### model_gk1 >>>')
# models['model_gk1'].summary()
# print('\n\n##################### model_user >>>')
# models['model_user'].summary()
# print('\n\n##################### model >>>')
# model = models['model']
# model.summary()

In [ ]:
num_features = 2

from sklearn import decomposition
pca = decomposition.PCA(n_components=num_features)
pca.fit(X_df)
rscore = pca.transform(X_df) / 5
print(rscore.shape)

#cscore = pca.components_.T / 3
cscore = np.zeros((0,num_features))
for icol in X_df:
    v = X_df[icol]
    tmp = rscore[v==1].mean(axis=0, keepdims=True)
    cscore = np.r_[cscore, tmp]
print(cscore.shape)

models = wd2v.make_model(num_features=num_features, rscore=rscore, cscore=cscore)

print('\n\n##################### model_gk1 >>>')
models['model_gk1'].summary()
print('\n\n##################### model_user >>>')
models['model_user'].summary()
print('\n\n##################### model >>>')
model = models['model']
model.summary()

In [ ]:
wgt_lm = wd2v.get_wgt_bycol()
print(wgt_lm.shape)
df = pd.DataFrame(wgt_lm[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)
save_img()

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
# wgt_user = model.get_layer('user_embedding').get_weights()[0]
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)
save_img()

### train

In [ ]:
%%time
hst, hst2 = wd2v.train(epochs=500, batch_size=32, verbose=0, lr0=0.001)
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()
save_img()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst2.history["loss"]))), hst2.history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst2.history["loss"]))), hst2.history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst2.history["loss"]))), hst2.history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()
save_img()

In [ ]:
wgt_prod = wd2v.get_wgt_bycol()
print(wgt_prod.shape)
df = pd.DataFrame(wgt_prod[:,:5])
df['brand_group'] = ['brand_group'+str(ii) for ii in X_df_brand_group['0'].values]
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist', hue='brand_group')
save_img()

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
df['brand_user_group'] = ['group' + str(ee) for ee in X_df_user_group['0'].values]
sns.pairplot(df, markers='o', hue='brand_user_group', height=3.5, diag_kind='hist',
             hue_order=['group'+str(ii) for ii in np.unique(X_df_user_group['0'].values)])
save_img()

In [ ]:
df1 = pd.DataFrame(wgt_prod)
df1['brand_group'] = X_df_brand_group['0'].values
df2 = pd.DataFrame(wgt_user)
df2['brand_group'] = ['user_group 1 and 7' if ee=='user_group1' or ee=='user_group7' else 'others' for ee in X_df_user_group['0'].values]
df = pd.concat([df2, df1])

plt.figure(figsize=(15,15))
ax = sns.kdeplot(data=df2, x=0, y=1,
                 levels=5, thresh=.1, hue='brand_group',
                 hue_order=['user_group 1 and 7', 'others'])
for ee in df1.iterrows():
    ax.scatter(ee[1][0], ee[1][1], s=200, marker='X', c='darkblue')
ax
save_img()

In [ ]:
df1 = pd.DataFrame(wgt_prod)
df1['brand_group'] = X_df_brand_group['0'].values
df2 = pd.DataFrame(wgt_user)
df2['brand_group'] = ['user_group 2 and 7' if ee=='user_group2' or ee=='user_group7' else 'others' for ee in X_df_user_group['0'].values]
df = pd.concat([df2, df1])

plt.figure(figsize=(15,15))
ax = sns.kdeplot(data=df2, x=0, y=1,
                 levels=5, thresh=.1, hue='brand_group',
                 hue_order=['user_group 2 and 7', 'others'])
for ee in df1.iterrows():
    ax.scatter(ee[1][0], ee[1][1], s=200, marker='X', c='darkblue')
ax
save_img()

In [ ]:
df1 = pd.DataFrame(wgt_prod)
df1['brand_group'] = X_df_brand_group['0'].values
df2 = pd.DataFrame(wgt_user)
df2['brand_group'] = ['user_group 1 and 7' if ee=='user_group1' or ee=='user_group7' else 'others' for ee in X_df_user_group['0'].values]
df = pd.concat([df2, df1])

plt.figure(figsize=(10,10))
ax = sns.kdeplot(data=df2, x=0, y=1,
                 levels=5, thresh=.1, hue='brand_group',
                 hue_order=['user_group 1 and 7', 'others'])
for ee in df1.iterrows():
    ax.scatter(ee[1][0], ee[1][1], s=200, marker='X', c='darkblue')
ax
save_img()

In [ ]:
df1 = pd.DataFrame(wgt_prod)
df1['brand_group'] = X_df_brand_group['0'].values
df2 = pd.DataFrame(wgt_user)
df2['brand_group'] = ['user_group 2 and 7' if ee=='user_group2' or ee=='user_group7' else 'others' for ee in X_df_user_group['0'].values]
df = pd.concat([df2, df1])

plt.figure(figsize=(10,10))
ax = sns.kdeplot(data=df2, x=0, y=1,
                 levels=5, thresh=.1, hue='brand_group',
                 hue_order=['user_group 2 and 7', 'others'])
for ee in df1.iterrows():
    ax.scatter(ee[1][0], ee[1][1], s=200, marker='X', c='darkblue')
ax
save_img()

In [ ]:
df_out_wgt_user = pd.DataFrame(wgt_user, index=X_df.index)
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.DataFrame(wgt_prod, index=X_df.columns)
df_out_wgt_col.head()

In [ ]:
df_out_wgt_user.to_csv('out_user.csv')
df_out_wgt_col.to_csv('out_col.csv')

In [ ]:
gamma = wd2v.get_gamma()
gamma, np.sqrt((1/gamma) / 2)

In [ ]:
np.save('out_gamma', gamma)